### 1. Obtain tweets

In [4]:
# Imports
import GetOldTweets3 as got
import pandas as pd

# Creation of query object
tweetCriteria = got.manager.TweetCriteria().setUsername("NCDCgov")\
                                           .setQuerySearch('new cases of')\
                                           .setSince("2020-05-19")\
                                           .setMaxTweets(1000)

# Creation of list that contains all tweets
tweets = got.manager.TweetManager.getTweets(tweetCriteria)

# Creating list of chosen tweet data
text_tweets = [[tweet.date, tweet.text,tweet.id,tweet.username] for tweet in tweets]

# Creation of dataframe from tweets
tweets_df = pd.DataFrame(text_tweets, columns = ['Datetime', 'Text','TweetID','username'])
tweets_df.head()
    
#     # Createspark spark dataframe
#     tweets_df_spark = spark.createDataFrame(tweets_df)

#     # Converting tweets dataframe to csv file
#     #tweets_df_spark.to_csv('C:\\Users\\brono\\First_batch\\Tweets19.csv',index = False,header=True)
#     tweets_df_spark.coalesce(1).write.save(path='C:\\Users\\brono\\First_batch\\Finalextract2.csv', format='csv', mode='append', inferSchema = True)
    
#     Finaldf = spark.read.csv("C:\\Users\\brono\\First_batch\\Finalextract2.csv", inferSchema = True, header = True)
#     Finaldf = Finaldf.dropDuplicates(subset=['TweetID'])
    
#     Finaldf.sort("TweetID").coalesce(1).write.mode("overwrite").option("header", "true").csv("C:\\Users\\brono\\First_batch\\Cleaned_data.csv")


,Datetime,Text,TweetID,username
0,2020-08-18 22:20:18+00:00,410 new cases of #COVID19Nigeria; Lagos-210 FC...,1295848020991451139,NCDCgov
1,2020-08-17 22:25:45+00:00,417 new cases of #COVID19Nigeria; Lagos-207 Ka...,1295487002675208194,NCDCgov
2,2020-08-16 22:27:47+00:00,298 new cases of #COVID19Nigeria; Plateau-108 ...,1295125127546507266,NCDCgov
3,2020-08-15 22:26:18+00:00,325 new cases of #COVID19Nigeria; Lagos-87 FCT...,1294762367734222849,NCDCgov
4,2020-08-14 22:49:22+00:00,"Our discharges today include 1,325 community r...",1294405785053732866,NCDCgov


### 2. Column structure

Temp column structure
- Datetime
- Year
- Month
- Day
- Text
- State
- New Cases
- Total Confirmed
- Total Discharged
- Total Dead

In [5]:
tweets_df.insert(loc=1, column='Day', value = pd.DatetimeIndex(tweets_df['Datetime']).day)
tweets_df.insert(loc=1, column='Month', value = pd.DatetimeIndex(tweets_df['Datetime']).month)
tweets_df.insert(loc=1, column='Year', value = pd.DatetimeIndex(tweets_df['Datetime']).year)

In [6]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   Datetime  100 non-null    datetime64[ns, UTC]
 1   Year      100 non-null    int64              
 2   Month     100 non-null    int64              
 3   Day       100 non-null    int64              
 4   Text      100 non-null    object             
 5   TweetID   100 non-null    object             
 6   username  100 non-null    object             
dtypes: datetime64[ns, UTC](1), int64(3), object(3)
memory usage: 5.6+ KB


#### 2.1 Split state data

In [87]:
states = [
  "Abia",
  "Adamawa",
  "Akwa Ibom",
  "Anambra",
  "Bauchi",
  "Bayelsa",
  "Benue",
  "Borno",
  "Cross River",
  "Delta",
  "Ebonyi",
  "Edo",
  "Ekiti",
  "Enugu",
  "FCT",
  "Gombe",
  "Imo",
  "Jigawa",
  "Kaduna",
  "Kano",
  "Katsina",
  "Kebbi",
  "Kogi",
  "Kwara",
  "Lagos",
  "Nasarawa",
  "Niger",
  "Ogun",
  "Ondo",
  "Osun",
  "Oyo",
  "Plateau",
  "Rivers",
  "Sokoto",
  "Taraba",
  "Yobe",
  "Zamfara"
]

Dict = {}
for stamp, tweet in zip (tweets_df.Datetime, tweets_df.Text):
# for tweet in tweets_df.Text:
    Dict[stamp] = {}
    #their count tweets start w the number of new cases
    if tweet[:3].isdigit():
        #their tweets usually start w #COVID19 or #COVID19Nigeria
        nextWhitePosHash = tweet.find(" ", tweet.find("#"), len(tweet))
        #remove the first bit before the state data
        tweet = tweet[nextWhitePosHash:len(tweet)]
        #for each state in the list of states
        for state in states:
            #find where the name of that state ends
            endChr = tweet.find(state) + len(state)
            #find next white space
            nextWhite = tweet.find(" ", endChr, len(tweet))
            if startChr == -1:
                Dict[stamp][state] = 0
            else:
                Dict[stamp][state] = tweet[endChr+1:nextWhite]
Dict

{Timestamp('2020-08-18 22:20:18+0000', tz='UTC'): {'Abia': 0,
  'Adamawa': 0,
  'Akwa Ibom': 0,
  'Anambra': 0,
  'Bauchi': 0,
  'Bayelsa': 0,
  'Benue': 0,
  'Borno': 0,
  'Cross River': 0,
  'Delta': 0,
  'Ebonyi': 0,
  'Edo': 0,
  'Ekiti': 0,
  'Enugu': 0,
  'FCT': 0,
  'Gombe': 0,
  'Imo': 0,
  'Jigawa': 0,
  'Kaduna': 0,
  'Kano': 0,
  'Katsina': 0,
  'Kebbi': 0,
  'Kogi': 0,
  'Kwara': 0,
  'Lagos': 0,
  'Nasarawa': 0,
  'Niger': 0,
  'Ogun': 0,
  'Ondo': 0,
  'Osun': 0,
  'Oyo': 0,
  'Plateau': 0,
  'Rivers': 0,
  'Sokoto': 0,
  'Taraba': 0,
  'Yobe': 0,
  'Zamfara': 0},
 Timestamp('2020-08-17 22:25:45+0000', tz='UTC'): {'Abia': 0,
  'Adamawa': 0,
  'Akwa Ibom': 0,
  'Anambra': 0,
  'Bauchi': 0,
  'Bayelsa': 0,
  'Benue': 0,
  'Borno': 0,
  'Cross River': 0,
  'Delta': 0,
  'Ebonyi': 0,
  'Edo': 0,
  'Ekiti': 0,
  'Enugu': 0,
  'FCT': 0,
  'Gombe': 0,
  'Imo': 0,
  'Jigawa': 0,
  'Kaduna': 0,
  'Kano': 0,
  'Katsina': 0,
  'Kebbi': 0,
  'Kogi': 0,
  'Kwara': 0,
  'Lagos': 0,
  '

In [97]:
df = pd.DataFrame.from_dict(Dict)
df = df.dropna(axis=1, how='all')
df.head()

,2020-08-18 22:20:18+00:00,2020-08-17 22:25:45+00:00,2020-08-16 22:27:47+00:00,2020-08-15 22:26:18+00:00,2020-08-14 22:42:06+00:00,2020-08-13 22:39:28+00:00,2020-08-12 22:29:55+00:00,2020-08-11 22:17:25+00:00,2020-08-10 22:21:00+00:00,2020-08-09 22:17:45+00:00,...,2020-05-28 22:27:18+00:00,2020-05-27 22:44:12+00:00,2020-05-26 22:30:43+00:00,2020-05-25 22:34:33+00:00,2020-05-24 22:10:08+00:00,2020-05-23 22:51:25+00:00,2020-05-22 22:35:01+00:00,2020-05-21 22:48:23+00:00,2020-05-20 22:20:41+00:00,2020-05-19 22:39:07+00:00
Abia,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Adamawa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Akwa Ibom,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Anambra,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bauchi,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37 entries, Abia to Zamfara
Data columns (total 100 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   2020-08-18 22:20:18+00:00  37 non-null     int64  
 1   2020-08-17 22:25:45+00:00  37 non-null     int64  
 2   2020-08-16 22:27:47+00:00  37 non-null     int64  
 3   2020-08-15 22:26:18+00:00  37 non-null     int64  
 4   2020-08-14 22:49:22+00:00  0 non-null      float64
 5   2020-08-14 22:42:06+00:00  37 non-null     int64  
 6   2020-08-13 22:39:28+00:00  37 non-null     int64  
 7   2020-08-12 22:29:55+00:00  37 non-null     int64  
 8   2020-08-11 22:17:25+00:00  37 non-null     int64  
 9   2020-08-10 22:21:00+00:00  37 non-null     int64  
 10  2020-08-09 22:17:45+00:00  37 non-null     int64  
 11  2020-08-08 22:21:40+00:00  37 non-null     int64  
 12  2020-08-07 22:25:49+00:00  37 non-null     int64  
 13  2020-08-06 22:32:12+00:00  37 non-null     int64

In [ ]:
df.to_csv('data/fromMay20.csv')

Plateau
108


417
new
cases
of
#COVID19Nigeria

Lagos
207
Kaduna
44
Ondo
38
Abia
28
Anambra
21
Plateau
20
Bauchi
13
Oyo
9
Ebonyi
9
Delta
7
Edo
7
Enugu
6
Niger
3
Gombe
2
Ogun
1
FCT
1
Kano
1
49,485
confirmed
36,834
discharged
977
deaths
298
new
cases
of
#COVID19Nigeria

Plateau
108
Kaduna
49
Lagos
47
Ogun
18
Osun
17
FCT
15
Ondo
14
Edo
8
Oyo
6
Akwa
Ibom
4
Cross
River
4
Borno
3
Ekiti
2
Bauchi
1
Kano
1
Rivers
1
49,068
confirmed
36,497
discharged
975
deaths
325
new
cases
of
#COVID19Nigeria

Lagos
87
FCT
49
Gombe
28
Ebonyi
20
Plateau
19
Kwara
18
Enugu
17
Imo
12
Rivers
12
Kaduna
11
Ogun
10
Edo
9
Oyo
9
Ondo
8
Osun
8
Ekiti
4
Borno
1
Kano
1
Bauchi
1
Nasarawa
1
48,770
confirmed
36,290
discharged
974
deaths
Our
discharges
today
include
1,325
community
recoveries
over
the
last
two
weeks
reported
from
Lagos
State
in
line
with
new
case
management
guidelines.
A
breakdown
of
cases
by
state
can
be
found
via
http://covid19.ncdc.gov.ng
#TakeResponsibility
329
new
cases
of
#COVID19Nigeria

Lagos
113
Kaduna
49
FCT
33
Plat

18
Edo
14
Bayelsa
13
Enugu
13
Osun
12
Kwara
10
Borno
8
Ogun
7
Kaduna
6
Imo
4
Bauchi
3
Gombe
3
Niger
2
Adamawa
1
30,249
confirmed
12,373
discharged
684
deaths
503
new
cases
of
COVID19Nigeria

Lagos
153
Ondo
76
Edo
54
FCT
41
Enugu
37
Rivers
30
Benue
24
Osun
20
Kaduna
15
Kwara
13
Abia
9
Borno
8
Plateau
6
Taraba
5
Ogun
3
Kano
3
Kebbi
2
Nasarawa
2
Bayelsa
1
Gombe
1
29,879
confirmed
12,108
discharged
669
deaths

575
new
cases
of
COVID19Nigeria

Lagos
123
FCT
100
Delta
58
Edo
52
Ogun
42
Katsina
24
Bayelsa
23
Rivers
22
Borno
19
Plateau
18
Ondo
18
Oyo
17
Kwara
15
Osun
13
Enugu
9
Nasarawa
7
Abia
6
Cross
River
5
Kaduna
3
Ekiti
1
29,286
confirmed
11,828
discharged
654
deaths

544
new
cases
of
#COVID19Nigeria

Lagos
199
Ebonyi
65
Oyo
47
Ondo
46
Ogun
31
Edo
30
FCT
28
Katsina
25
Plateau
15
Bayelsa
11
Kaduna
10
Adamawa
10
Akwa
Ibom
8
Gombe
7
Kano
4
Taraba
3
Rivers
2
Abia
2
Ekiti
1
28,711
confirmed
11,665
discharged
645
deaths
603
new
cases
of
#COVID19Nigeria

Lagos
135
Edo
87
FCT
73
Rivers
67
Delta
62

416
new
cases
of
#COVID19

Lagos
192
Edo
41
Rivers
33
Kaduna
30
Kwara
23
Nasarawa
18
Borno
17
FCT
14
Oyo
10
Katsina
7
Abia
5
Delta
5
Adamawa
4
Kano
4
Imo
3
Ondo
3
Benue
2
Bauchi
2
Ogun
2
Niger
1
10578
cases
of
#COVID19Nigeria
Discharged:
3122
Deaths:
299
#TakeResponsibility
307
new
cases
of
#COVID19

Lagos
188
FCT
44
Ogun
19
Kaduna
14
Oyo
12
Bayelsa
9
Gombe
5
Kano
3
Delta
3
Imo
2
Rivers
2
Niger
2
Bauchi
2
Plateau
1
Kwara
1
10162
cases
of
#COVID19Nigeria
Discharged:
3007
Deaths:
287
#TakeResponsibility
553
new
cases
of
#COVID19

Lagos
378
FCT
52
Delta
23
Edo
22
Rivers
14
Ogun
13
Kaduna
12
Kano
9
Borno
7
Katsina
6
Jigawa
5
Oyo
5
Yobe
3
Plateau
3
Osun
1
9855
cases
of
#COVID19Nigeria
Discharged:
2856
Deaths:
273
387
new
cases
of
#COVID19

Lagos
254
FCT
29
Jigawa
24
Edo
22
Oyo
15
Rivers
14
Kaduna
11
Borno
6
Kano
3
Plateau
2
Yobe
2
Gombe
2
Bauchi
2
Ondo
1
9302
cases
of
#COVID19Nigeria
Discharged:
2697
Deaths:
261
182
new
cases
of
#COVID19

Lagos
111
FCT
16
Akwa
Ibom
10
Oyo
8
Kaduna
6
Delta
6

confirmed
cases
of
#COVID19
in
Nigeria
Discharged:
319
Deaths:
58
196
new
cases
of
#COVID19
reported

87
Lagos
24
Kano
18
Gombe
17
Kaduna
16
FCT
10
Katsina
8
Sokoto
7
Edo
6
Borno
1
Yobe
1
Ebonyi
1
Adamawa
As
at
11:55pm
29th
April

1728
confirmed
cases
of
#COVID19
reported
in
Nigeria.
Discharged:
307
Deaths:
51
195
new
cases
of
#COVID19
reported

80
Lagos
38
Kano
15
Ogun
15
Bauchi
11
Borno
10
Gombe
9
Sokoto
5
Edo
5
Jigawa
2
Zamfara
1
Rivers
1
Enugu
1
Delta
1
FCT
1
Nasarawa
As
at
11:50pm
28th
April

1532
confirmed
cases
of
#COVID19
reported
in
Nigeria.
Discharged:
255
Deaths:
44
64
new
cases
of
#COVID19
reported

34
Lagos
15
FCT
11
Borno
2
Taraba
2
Gombe
As
at
11:20pm
27th
April

1337
confirmed
cases
of
#COVID19
reported
in
Nigeria.
Discharged:
255
Deaths:
40
"In
Kano
&
many
other
States
recording
new
cases,
preliminary
findings
show
that
such
cases
are
mostly
from
interstate
travel
and
emerging
community
transmission
I
implore
all
Nigerians
to
continue
to
adhere
strictly
to
advisories
p

been
reported
in
Nigeria

9
in
Osun,
2
in
Edo
and
1
in
Ekiti
State
As
at
12:30
pm
1st
April
there
are
151
confirmed
cases
of
#COVID19
reported
in
Nigeria.
Nine
have
been
discharged
with
two
deaths
Four
new
cases
of
#COVID19
have
been
reported
in
Nigeria

3
in
the
FCT
and
1
in
Lagos
As
at
08:00
pm
31st
March
there
are
139
confirmed
cases
of
#COVID19
reported
in
Nigeria.
Nine
have
been
discharged
with
two
deaths
Four
new
cases
of
#COVID19
have
been
reported
in
Nigeria

3
in
Osun
and
1
in
Ogun
As
at
11:15
am
31st
March
there
are
135
confirmed
cases
of
#COVID19
reported
in
Nigeria
with
2
deaths
Twenty
new
cases
of
#COVID19
have
been
reported
in
Nigeria

13
in
Lagos,
4
in
FCT,
2
in
Kaduna
and
1
in
Oyo
State
As
at
09:00
pm
30th
March
there
are
131
confirmed
cases
of
#COVID19
reported
in
Nigeria
with
2
deaths
Fourteen
new
cases
of
#COVID19
have
been
reported
in
Nigeria

9
in
Lagos
and
5
in
FCT
As
at
09:30
pm
29th
March
there
are
111
confirmed
cases
of
#COVID19
reported
in
Nigeria
with
1
death

@SORMAS_open
platform.
We
are
strengthening
capacity
for
effective
use
of
our
new
digital
infrastructure
for
disease
surveillance.

Our
#LassaFever
Case
Management
team
is
in
Ondo
State
to
train
staff
at
the
Federal
Medical
Centre
in
Owo
on
case
management
documentation
using
the
@SORMAS_open
platform.
We
are
strengthening
capacity
for
effective
use
of
our
new
digital
infrastructure
for
disease
surveillance.

The
latest
situation
report
on
#LassaFever
has
been
published
on
the
NCDC
website.
We
have
recorded
a
decline
in
the
number
of
new
cases
and
in
the
number
of
cases
currently
admitted
across
treatment
facilities.
View
and
download
the
report
via:
https://ncdc.gov.ng/diseases/sitreps/?cat=5&amp
name=An%20update%20of%20Lassa%20fever%20outbreak%20in%20Nigeria
NEW
PUBLICATION!
Paper
in
@CDC_EIDjournal
by
NCDC
&amp

partners
describes
the
increase
in
#Lassafever
cases
in
2018
&amp

concludes
that
the
most
likely
explanation
was

addition
of
new
laboratories,
intensified
response
&amp

i

'298 new cases of #COVID19Nigeria; Plateau-108 Kaduna-49 Lagos-47 Ogun-18 Osun-17 FCT-15 Ondo-14 Edo-8 Oyo-6 Akwa Ibom-4 Cross River-4 Borno-3 Ekiti-2 Bauchi-1 Kano-1 Rivers-1 49,068 confirmed 36,497 discharged 975 deaths'

,Datetime,Text,TweetID,username,geo,Year
0,2020-08-17 22:25:45+00:00,417 new cases of #COVID19Nigeria; Lagos-207 Ka...,1295487002675208194,NCDCgov,,2020
1,2020-08-16 22:27:47+00:00,298 new cases of #COVID19Nigeria; Plateau-108 ...,1295125127546507266,NCDCgov,,2020
2,2020-08-15 22:26:18+00:00,325 new cases of #COVID19Nigeria; Lagos-87 FCT...,1294762367734222849,NCDCgov,,2020
3,2020-08-14 22:49:22+00:00,"Our discharges today include 1,325 community r...",1294405785053732866,NCDCgov,,2020
4,2020-08-14 22:42:06+00:00,329 new cases of #COVID19Nigeria; Lagos-113 Ka...,1294403954307141633,NCDCgov,,2020


,Datetime,A,Text,TweetID,username,geo,Year,Month,Day
0,2020-08-17 22:25:45+00:00,2020,417 new cases of #COVID19Nigeria; Lagos-207 Ka...,1295487002675208194,NCDCgov,,2020,8,17
1,2020-08-16 22:27:47+00:00,2020,298 new cases of #COVID19Nigeria; Plateau-108 ...,1295125127546507266,NCDCgov,,2020,8,16
2,2020-08-15 22:26:18+00:00,2020,325 new cases of #COVID19Nigeria; Lagos-87 FCT...,1294762367734222849,NCDCgov,,2020,8,15
3,2020-08-14 22:49:22+00:00,2020,"Our discharges today include 1,325 community r...",1294405785053732866,NCDCgov,,2020,8,14
4,2020-08-14 22:42:06+00:00,2020,329 new cases of #COVID19Nigeria; Lagos-113 Ka...,1294403954307141633,NCDCgov,,2020,8,14


,Datetime,Text,TweetID,username,geo,Year,Month
0,2020-08-17 22:25:45+00:00,417 new cases of #COVID19Nigeria; Lagos-207 Ka...,1295487002675208194,NCDCgov,,2020,8
1,2020-08-16 22:27:47+00:00,298 new cases of #COVID19Nigeria; Plateau-108 ...,1295125127546507266,NCDCgov,,2020,8
2,2020-08-15 22:26:18+00:00,325 new cases of #COVID19Nigeria; Lagos-87 FCT...,1294762367734222849,NCDCgov,,2020,8
3,2020-08-14 22:49:22+00:00,"Our discharges today include 1,325 community r...",1294405785053732866,NCDCgov,,2020,8
4,2020-08-14 22:42:06+00:00,329 new cases of #COVID19Nigeria; Lagos-113 Ka...,1294403954307141633,NCDCgov,,2020,8


,Datetime,Text,TweetID,username,geo,Year,Month,Day
0,2020-08-17 22:25:45+00:00,417 new cases of #COVID19Nigeria; Lagos-207 Ka...,1295487002675208194,NCDCgov,,2020,8,17
1,2020-08-16 22:27:47+00:00,298 new cases of #COVID19Nigeria; Plateau-108 ...,1295125127546507266,NCDCgov,,2020,8,16
2,2020-08-15 22:26:18+00:00,325 new cases of #COVID19Nigeria; Lagos-87 FCT...,1294762367734222849,NCDCgov,,2020,8,15
3,2020-08-14 22:49:22+00:00,"Our discharges today include 1,325 community r...",1294405785053732866,NCDCgov,,2020,8,14
4,2020-08-14 22:42:06+00:00,329 new cases of #COVID19Nigeria; Lagos-113 Ka...,1294403954307141633,NCDCgov,,2020,8,14


False

0      417 new cases of #COVID19Nigeria; Lagos-207 Ka...
1      298 new cases of #COVID19Nigeria; Plateau-108 ...
2      325 new cases of #COVID19Nigeria; Lagos-87 FCT...
4      329 new cases of #COVID19Nigeria; Lagos-113 Ka...
5      373 new cases of #COVID19Nigeria; Lagos-69 Osu...
                             ...                        
184    Three new cases of #COVID19 have been confirme...
187    @Fmohnigeria has confirmed 10 new cases of #CO...
188    Today, 4 new cases of #COVID19 were confirmed ...
190    @Fmohnigeria has announced 5 new cases of #COV...
196    “Once there’s a new virus such as #COVID19, th...
Name: Text, Length: 166, dtype: object

### PDF alternative

In [1]:
!java -version

java version "14.0.2" 2020-07-14
Java(TM) SE Runtime Environment (build 14.0.2+12-46)
Java HotSpot(TM) 64-Bit Server VM (build 14.0.2+12-46, mixed mode, sharing)


In [2]:
!pip install -q tabula-py

In [3]:
import tabula

tabula.environment_info()

Python version:
    3.8.3 (default, Jul  2 2020, 11:26:31) 
[Clang 10.0.0 ]
Java version:
    java version "14.0.2" 2020-07-14
Java(TM) SE Runtime Environment (build 14.0.2+12-46)
Java HotSpot(TM) 64-Bit Server VM (build 14.0.2+12-46, mixed mode, sharing)
tabula-py version: 2.1.1
platform: macOS-10.15.6-x86_64-i386-64bit
uname:
    uname_result(system='Darwin', node='Tibers-mbp.lan', release='19.6.0', version='Darwin Kernel Version 19.6.0: Sun Jul  5 00:43:10 PDT 2020; root:xnu-6153.141.1~9/RELEASE_X86_64', machine='x86_64', processor='i386')
linux_distribution: ('Darwin', '19.6.0', '')
mac_ver: ('10.15.6', ('', '', ''), 'x86_64')
    


In [13]:
pdf_path = "data/test2.pdf"

dfs = tabula.read_pdf(pdf_path, stream=True, pages="2")
# read_pdf returns list of DataFrames
print(len(dfs))
dfs[0]

1


,Unnamed: 0,CONFIRMED CASES,DISCHARGED CASES,DEATHS,Unnamed: 1,DAYS SINCE
0,NaN,NaN,NaN,NaN,TOTAL,NaN
1,NaN,NaN,NaN,NaN,NaN,LAST
2,STATES,NaN,NaN,NaN,ACTIVE,NaN
3,NaN,NaN,NaN,NaN,NaN,REPORTED
4,NaN,TOTAL NEW,TOTAL NEW,TOTAL NEW,CASES,CASE


In [14]:
pip install camelot-py[cv]

     |████████████████████████████████| 42 kB 445 kB/s eta 0:00:01
     |████████████████████████████████| 5.6 MB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 52.2 MB 466 kB/s eta 0:00:01
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61084 sha256=7bacf05111491d78471e1487f97a10cb95ad9d285536388603cf6f2025c321a5
  Stored in directory: /Users/tiberb/Library/Caches/pip/wheels/b1/1a/8f/a4c34be976825a2f7948d0fa40907598d69834f8ab5889de11
Successfully built PyPDF2
Note: you may need to restart the kernel to use updated packages.


In [1]:
import camelot
>>> tables = camelot.read_pdf('data/test2.pdf')
>>> tables

RuntimeError: Please make sure that Ghostscript is installed

In [2]:
pip install ghostscript

Note: you may need to restart the kernel to use updated packages.


In [2]:
# extract info from all pdf files into one
# all in one loop
import pdfplumber
from glob import glob
# get pdf file names
files = glob('data/*.pdf')
# set up a df where the output will go
column_names = ['state', 'total_confirmed', 'new_confirmed', 'total_discharged', 'new_dicharged',
                'total_deaths', 'new_deaths', 'total_active', 'days_since_last_case', 'date']
pdf_df = pd.DataFrame(columns=column_names)
# loop through all files
for file in files:
    # just to keep track
    print(file)
    # extract date from the title
    pdf_date = file.split('/')[-1].split('_')[0]
    #print(pdf_date)
    # read pdf file, extract text from the page with the table
    pdf = pdfplumber.open(file)
    table_page = pdf.pages[1]
    # split table page into lines
    page_text = table_page.extract_text()
    page_lines = page_text.split('\n')
    # establish where the table text starts & finishes
    # then save table text into a new variable
    counter_start = 0
    counter_stop = 0
    for i, line in enumerate(page_lines):
        if line.startswith('CASE'):
            counter_start = i + 1
        if line.startswith('States'):
            counter_stop = i -1
    # print (counter_start, counter_stop)
    table_text = page_lines[counter_start:counter_stop]
    # convert text from the table into a pandas df
    for line in table_text:
        words = line.split()
        line_dict = {}
        line_dict['date'] = pdf_date
        for i, column in enumerate(column_names[:-1]): # ignore the 'date' column
            line_dict[column] = words[i]
        pdf_df = pdf_df.append(line_dict, ignore_index=True)

ModuleNotFoundError: No module named 'pdfplumber'